In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import mysql.connector as sql

db_connection = sql.connect(host='127.0.0.1', database='mydatabase', user='root', password='12345')
air=pd.read_sql("select * from air",con=db_connection)

In [3]:
air.shape

(80, 10)

In [4]:
air.head()

,index,Week_num,Passengers,Promotion_Budget,Service_Quality_Score,Holiday_week,Delayed_Cancelled_flight_ind,Inter_metro_flight_ratio,Bad_Weather_Ind,Technical_issues_ind
0,0,1,37824,517356,4.00000,NO,NO,0.70,YES,YES
1,1,2,43936,646086,2.67466,NO,YES,0.80,YES,YES
2,2,3,42896,638330,3.29473,NO,NO,0.90,NO,NO
3,3,4,35792,506492,3.85684,NO,NO,0.40,NO,NO
4,4,5,38624,609658,3.90757,NO,NO,0.87,NO,YES


In [5]:
air.drop(["index"],axis=1,inplace=True)

In [6]:
air.head(2)

,Week_num,Passengers,Promotion_Budget,Service_Quality_Score,Holiday_week,Delayed_Cancelled_flight_ind,Inter_metro_flight_ratio,Bad_Weather_Ind,Technical_issues_ind
0,1,37824,517356,4.00000,NO,NO,0.7,YES,YES
1,2,43936,646086,2.67466,NO,YES,0.8,YES,YES


In [7]:
air.dtypes

Week_num                          int64
Passengers                        int64
Promotion_Budget                  int64
Service_Quality_Score           float64
Holiday_week                     object
Delayed_Cancelled_flight_ind     object
Inter_metro_flight_ratio        float64
Bad_Weather_Ind                  object
Technical_issues_ind             object
dtype: object

In [11]:
air_num=air.select_dtypes(include=[np.number]).copy()

In [12]:
air_num.head(2)

,Week_num,Passengers,Promotion_Budget,Service_Quality_Score,Inter_metro_flight_ratio
0,1,37824,517356,4.00000,0.7
1,2,43936,646086,2.67466,0.8


In [13]:
air_num.drop(["Week_num"],axis=1,inplace=True)

In [14]:
air_num.head(2)

,Passengers,Promotion_Budget,Service_Quality_Score,Inter_metro_flight_ratio
0,37824,517356,4.00000,0.7
1,43936,646086,2.67466,0.8


# exract the depdent variable and indepdent variable

In [16]:
x_train=air_num.drop(["Passengers"],axis=1)
y_train=air_num["Passengers"]

In [17]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(x_train,y_train)

LinearRegression()

In [18]:
lr.score(x_train,y_train)

0.9513650522199228

In [19]:
lr.coef_

array([ 5.55027075e-02, -2.80207080e+03, -2.00345080e+03])

In [20]:
list(zip(x_train.columns,lr.coef_))

[('Promotion_Budget', 0.05550270754241695),
 ('Service_Quality_Score', -2802.070802434244),
 ('Inter_metro_flight_ratio', -2003.4508001029658)]

In [21]:
lr.intercept_

19213.811570876966

In [23]:
air["pred"]=lr.intercept_+air["Promotion_Budget"]*0.05550270754241695+air["Service_Quality_Score"]*-2802.070802434244 \
            +air["Inter_metro_flight_ratio"]*-2003.4508001029658

In [24]:
air.head(2)

,Week_num,Passengers,Promotion_Budget,Service_Quality_Score,Holiday_week,Delayed_Cancelled_flight_ind,Inter_metro_flight_ratio,Bad_Weather_Ind,Technical_issues_ind,pred
0,1,37824,517356,4.00000,NO,NO,0.7,YES,YES,35317.771564
1,2,43936,646086,2.67466,NO,YES,0.8,YES,YES,45975.986544


In [25]:
x_train.head(2)

,Promotion_Budget,Service_Quality_Score,Inter_metro_flight_ratio
0,517356,4.00000,0.7
1,646086,2.67466,0.8


In [28]:
import mysql.connector

mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  passwd="12345",
  database="mydatabase"
)

mycursor = mydb.cursor()

mycursor.execute("CREATE TABLE air_3 (Promotion_Budget INT, Service_Quality_Score INT,Inter_metro_flight_ratio INT )")

In [30]:
import mysql.connector
mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  passwd="12345",
  database="mydatabase"
)
mycursor = mydb.cursor()
sql = "INSERT INTO air_3 (Promotion_Budget, Service_Quality_Score,Inter_metro_flight_ratio) VALUES (%s, %s,%s)"
val = [
    (12000,4,3),
    (14000,1,2)
   
]
mycursor.executemany(sql, val)
mydb.commit()
print(mycursor.rowcount, "record(s) inserted.")

2 record(s) inserted.


In [31]:
lr.coef_

array([ 5.55027075e-02, -2.80207080e+03, -2.00345080e+03])

In [32]:
lr.intercept_

19213.811570876966

In [34]:
import os
os.getcwd()

'C:\\Users\\Koti\\Documents\\Data_Science\\DS_APR_2020'

In [35]:
path=r"C:\Users\Koti\Documents\Data_Science\DS_APR_2020\model_deploy"
os.chdir(path)
os.getcwd()

'C:\\Users\\Koti\\Documents\\Data_Science\\DS_APR_2020\\model_deploy'

In [39]:
import joblib

joblib.dump(lr,"model.pkl")
lr1= joblib.load("model.pkl")
model_columns = list(x_train.columns)
joblib.dump(model_columns,"Model_columns.pkl")

['Model_columns.pkl']

In [40]:
lr1.coef_

array([ 5.55027075e-02, -2.80207080e+03, -2.00345080e+03])

In [41]:
from flask import Flask ,jsonify,request
import joblib
import traceback
import pandas as pd
import numpy as np



app = Flask(__name__)


@app.route("/predict", methods=["POST","GET"])
def predict():
    if ler:
        try:
            json_ = request.json
            query = pd.get_dummies(pd.DataFrame(json_))
            query = query.reindex(columns= model_columns,fill_value=0)

            prediction = list(ler.predict(query))
            return jsonify({"prediction":str(prediction)})
        except:
            return jsonify({"trace":traceback.format_exc()})
    else:
        print("Train the  model first")
        return "No model is here to use"

if __name__ == '__main__':
    try:
        port = int(sys.argv[1])
    except:
        port =12345
    ler = joblib.load("model.pkl")
    print("Model Load Successfully")
    model_columns = joblib.load("Model_columns.pkl")
    print("Model columns load Successfully")
    app.run(debug=False,port=port)


<IPython.core.display.Javascript object>

Model Load Successfully
Model columns load Successfully
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:12345/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Aug/2020 08:50:16] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [08/Aug/2020 08:50:16] "GET /favicon.ico HTTP/1.1" 404 -


In [42]:
x_train.columns

Index(['Promotion_Budget', 'Service_Quality_Score',
       'Inter_metro_flight_ratio'],
      dtype='object')

In [43]:
x_test=pd.DataFrame({"Promotion_Budget":[10],"Service_Quality_Score":[3],"Inter_metro_flight_ratio":[7]})
x_test

,Promotion_Budget,Service_Quality_Score,Inter_metro_flight_ratio
0,10,3,7


In [44]:
x_test["pred"]=lr.predict(x_test)

In [45]:
x_test

,Promotion_Budget,Service_Quality_Score,Inter_metro_flight_ratio,pred
0,10,3,7,-3216.00141
